# 🧠 INVESTIGACIÓN: Aplicación de Operaciones Matriciales en RNN para NLP

**Tema:** Redes Neuronales Recurrentes en Procesamiento de Lenguaje Natural
**Aplicación:** Clasificación de Sentimientos (Positivo/Negativo)

Este cuaderno implementa un clasificador RNN simple **desde cero** utilizando `numpy` para:

1.  Demostrar explícitamente las **operaciones matriciales** fundamentales (multiplicación de matrices) en el _forward_ y _backward_ pass.
2.  Entrenar el modelo con un pequeño conjunto de datos de reseñas de películas.
3.  Analizar la **complejidad computacional** de estas operaciones.

---
**Arquitectura RNN Clave:**
$$\mathbf{h}_t = \tanh(\mathbf{x}_t \mathbf{W}_{xh} + \mathbf{h}_{t-1} \mathbf{W}_{hh} + \mathbf{b}_h)$$

* $\mathbf{x}_t \mathbf{W}_{xh}$: Proyección de la **entrada actual**.
* $\mathbf{h}_{t-1} \mathbf{W}_{hh}$: Proyección del **estado oculto anterior** (la recurrencia).
* Ambas son **multiplicaciones matriciales** de alta densidad, clave para el rendimiento en hardware vectorizado (CPU/GPU). 

In [ ]:
import numpy as np
from collections import defaultdict
import random
from typing import Tuple, List
import sys

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

class Config:
	"""Parámetros del modelo."""
	VOCAB_SIZE = 200
	EMBEDDING_DIM = 32
	HIDDEN_DIM = 16
	OUTPUT_DIM = 2 # 0: Negativo, 1: Positivo
	LEARNING_RATE = 0.05 # Reducido un poco para mayor estabilidad en el ejemplo
	EPOCHS = 20 # Aumentado un poco para mejor convergencia
	BATCH_SIZE = 1
	MAX_SEQUENCE_LEN = 30
	
	# Visualización
	VERBOSE = True
	PRINT_EVERY = 5


# ============================================================================
# CLASE PRINCIPAL: RNN IMPLEMENTACIÓN MANUAL
# ============================================================================

class SimpleRNNClassifier:
	"""
	Clasificador RNN manual que demuestra explícitamente las operaciones
	matriciales fundamentales en redes neuronales recurrentes.
	"""
	
	def __init__(self, config: Config):
		self.config = config
		self.initialize_parameters()
		
	def initialize_parameters(self):
		"""Inicializar matrices de peso con Xavier/He initialization (aproximación)."""
		# Xavier initialization: var = 1 / n_in
		xavier_embedding = np.sqrt(1.0 / self.config.EMBEDDING_DIM)
		xavier_hidden = np.sqrt(1.0 / self.config.HIDDEN_DIM)
		
		# Matriz de Embedding (VOCAB_SIZE × EMBEDDING_DIM)
		self.W_embedding = np.random.randn(
			self.config.VOCAB_SIZE, 
			self.config.EMBEDDING_DIM
		) * 0.01
		
		# Matriz entrada → oculto (EMBEDDING_DIM × HIDDEN_DIM)
		self.W_xh = np.random.randn(
			self.config.EMBEDDING_DIM,
			self.config.HIDDEN_DIM
		) * xavier_embedding
		
		# Matriz oculto → oculto (HIDDEN_DIM × HIDDEN_DIM)
		self.W_hh = np.random.randn(
			self.config.HIDDEN_DIM,
			self.config.HIDDEN_DIM
		) * xavier_hidden
		
		# Matriz oculto → salida (HIDDEN_DIM × OUTPUT_DIM)
		self.W_hy = np.random.randn(
			self.config.HIDDEN_DIM,
			self.config.OUTPUT_DIM
		) * xavier_hidden
		
		# Sesgos
		self.b_h = np.zeros((1, self.config.HIDDEN_DIM))
		self.b_y = np.zeros((1, self.config.OUTPUT_DIM))
		
	def forward(self, word_indices: List[int]) -> Tuple[np.ndarray, List, List]:
		"""
		Forward pass (propagación hacia adelante).
		OPERACIONES MATRICIALES CLAVE:
		1. x_t @ W_xh
		2. h_{t-1} @ W_hh
		3. h_t @ W_hy
		"""
		h_t = np.zeros((1, self.config.HIDDEN_DIM))
		h_sequence = [h_t.copy()]
		x_sequence = []
		
		# Procesar cada palabra en la secuencia (pasos temporales)
		for word_idx in word_indices:
			# PASO 1: Embedding Lookup (similar a una multiplicación matricial One-Hot @ W_embedding)
			x_t = self.W_embedding[word_idx:word_idx+1, :]
			x_sequence.append(x_t)
			
			# PASO 2: OPERACIÓN MATRICIAL - Transformación de entrada
			# (1 × embedding_dim) × (embedding_dim × hidden_dim) = (1 × hidden_dim)
			h_input = x_t @ self.W_xh
			
			# PASO 3: OPERACIÓN MATRICIAL - Transformación recurrente (memoria)
			# (1 × hidden_dim) × (hidden_dim × hidden_dim) = (1 × hidden_dim)
			h_recurrent = h_t @ self.W_hh
			
			# PASO 4: Combinación lineal + activación
			z = h_input + h_recurrent + self.b_h
			h_t = np.tanh(z)
			h_sequence.append(h_t.copy())
		
		# PASO 5: OPERACIÓN MATRICIAL - Proyección a espacio de salida (Clasificación)
		# (1 × hidden_dim) × (hidden_dim × output_dim) = (1 × output_dim)
		logits = h_t @ self.W_hy + self.b_y
		
		return logits, h_sequence, x_sequence
	
	def backward(self, word_indices: List[int], h_sequence: List,
				 x_sequence: List, logits: np.ndarray, label: int):
		"""
		Backward pass: Backpropagation Through Time (BPTT).
		Implica multiplicación matricial con la transpuesta (e.g., x.T @ delta) para calcular gradientes.
		"""
		# Error de predicción
		probs = self._softmax(logits)
		delta_y = probs.copy()
		delta_y[0, label] -= 1 # Derivada de Cross-Entropy + Softmax
		
		# Gradientes de capa de salida
		# (hidden_dim × 1) × (1 × output_dim) = (hidden_dim × output_dim)
		dW_hy = h_sequence[-1].T @ delta_y 
		db_y = delta_y
		
		# Inicializar acumuladores de gradientes
		dW_xh = np.zeros_like(self.W_xh)
		dW_hh = np.zeros_like(self.W_hh)
		db_h = np.zeros_like(self.b_h)
		dW_embedding = np.zeros_like(self.W_embedding)
		
		# Error retropropagado a capa oculta (delta_h)
		delta_h = delta_y @ self.W_hy.T
		
		# Backprop a través de cada paso temporal (BPTT - tiempo invertido)
		for t in range(len(word_indices) - 1, -1, -1):
			word_idx = word_indices[t]
			
			# Aplicar derivada de tanh
			d_tanh = 1 - h_sequence[t+1] ** 2
			delta_h_raw = delta_h * d_tanh # Element-wise multiplication
			
			# Gradientes de matrices (OPERACIONES MATRICIALES CLAVE: Transposición + Multiplicación)
			# dW_xh: (emb_dim × 1) × (1 × hidden_dim) = (emb_dim × hidden_dim)
			dW_xh += x_sequence[t].T @ delta_h_raw 
			# dW_hh: (hidden_dim × 1) × (1 × hidden_dim) = (hidden_dim × hidden_dim)
			dW_hh += h_sequence[t].T @ delta_h_raw 
			db_h += delta_h_raw
			
			# Gradiente del embedding
			dW_embedding[word_idx, :] += (delta_h_raw @ self.W_xh.T).flatten()
			
			# Propagar gradiente al paso anterior (recurrencia)
			delta_h = delta_h_raw @ self.W_hh.T
			
		# Actualizar parámetros (descenso de gradiente)
		self.W_xh -= self.config.LEARNING_RATE * dW_xh
		self.W_hh -= self.config.LEARNING_RATE * dW_hh
		self.W_hy -= self.config.LEARNING_RATE * dW_hy
		self.b_h -= self.config.LEARNING_RATE * db_h
		self.b_y -= self.config.LEARNING_RATE * db_y
		self.W_embedding -= self.config.LEARNING_RATE * dW_embedding
	
	def predict(self, word_indices: List[int]) -> Tuple[int, float, np.ndarray]:
		"""Hacer predicción en una secuencia."""
		logits, _, _ = self.forward(word_indices)
		probs = self._softmax(logits)
		pred_class = np.argmax(probs[0])
		confidence = probs[0, pred_class]
		return pred_class, confidence, probs[0]
	
	@staticmethod
	def _softmax(x: np.ndarray) -> np.ndarray:
		"""Softmax estable numéricamente."""
		exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
		return exp_x / np.sum(exp_x, axis=1, keepdims=True)
	
	def calculate_loss(self, logits: np.ndarray, label: int) -> float:
		"""Calcular cross-entropy loss."""
		probs = self._softmax(logits)
		return -np.log(probs[0, label] + 1e-10)


# ============================================================================
# UTILIDADES: PROCESAMIENTO DE TEXTO
# ============================================================================

class TextPreprocessor:
	"""Construir vocabulario y codificar texto."""
	
	def __init__(self, max_vocab: int = 500):
		self.max_vocab = max_vocab
		self.word2idx = {'<PAD>': 0, '<UNK>': 1}
		self.idx2word = {0: '<PAD>', 1: '<UNK>'}
		self.word_freq = defaultdict(int)
		
	def build_vocab(self, texts: List[str]):
		"""Construir vocabulario a partir de textos."""
		for text in texts:
			for word in text.lower().split():
				self.word_freq[word] += 1
		
		# Añadir palabras más frecuentes
		sorted_words = sorted(self.word_freq.items(), 
								 key=lambda x: x[1], reverse=True)
		
		for idx, (word, freq) in enumerate(sorted_words[:self.max_vocab-2], 2):
			self.word2idx[word] = idx
			self.idx2word[idx] = word
	
	def encode(self, text: str, max_len: int = None) -> List[int]:
		"""Convertir texto a índices."""
		if max_len is None:
			max_len = 30
		
		words = text.lower().split()[:max_len]
		indices = [self.word2idx.get(w, 1) for w in words] # 1 es <UNK>
		
		# Padding (relleno)
		if len(indices) < max_len:
			indices += [0] * (max_len - len(indices)) # 0 es <PAD>
		
		return indices[:max_len]
	
	def decode(self, indices: List[int]) -> str:
		"""Convertir índices a texto."""
		return ' '.join([self.idx2word.get(idx, '<UNK>') for idx in indices])


## 📚 Preparación de Datos y Configuración

In [ ]:
# 1. CONFIGURACIÓN
config = Config()
print("📊 [1/5] CONFIGURACIÓN DEL MODELO")
print("-" * 80)
print(f"  ✓ Tamaño vocabulario: {config.VOCAB_SIZE}")
print(f"  ✓ Dimensión embedding: {config.EMBEDDING_DIM}")
print(f"  ✓ Dimensión oculta: {config.HIDDEN_DIM}")
print(f"  ✓ Clases de salida: {config.OUTPUT_DIM}")
print(f"  ✓ Épocas entrenamiento: {config.EPOCHS}")
print(f"  ✓ Tasa de aprendizaje: {config.LEARNING_RATE}")
print("\n")

# 2. PREPARACIÓN DE DATOS (Dataset de ejemplo)
print("📝 [2/5] PREPARACIÓN DE DATOS")
print("-" * 80)
positive_reviews = [
	"esta película es absolutamente excelente muy buena",
	"obra maestra del cine increíble",
	"actores fantásticos historia fascinante",
	"mejor película del año recomendada",
	"simplemente magnífica y emocionante",
	"adoré cada segundo película perfecta",
	"cinematografía hermosa actuaciones brillantes",
	"me encantó totalmente genial",
]
negative_reviews = [
	"película horrible muy aburrida mala",
	"pérdida completa de tiempo",
	"actores terribles historia débil",
	"muy mala no recomendada",
	"decepcionante y frustrante",
	"basura no aguanté ver",
	"actuaciones malas guión pésimo",
	"no entiendo por qué la hacen",
]

all_texts = positive_reviews + negative_reviews
all_labels = [1] * len(positive_reviews) + [0] * len(negative_reviews) # 1: Positivo, 0: Negativo

print(f"  ✓ Total muestras: {len(all_texts)}\n")

# 3. CONSTRUCCIÓN DEL VOCABULARIO
print("🔤 [3/5] CONSTRUCCIÓN DEL VOCABULARIO")
print("-" * 80)
preprocessor = TextPreprocessor(max_vocab=config.VOCAB_SIZE)
preprocessor.build_vocab(all_texts)

print(f"  ✓ Palabras únicas en dataset: {len(preprocessor.word2idx)}\n")

# Codificar textos
encoded_texts = [preprocessor.encode(text, max_len=config.MAX_SEQUENCE_LEN) 
						 for text in all_texts]

print(f"  ✓ Ejemplo de codificación (Primer Review Positivo):")
print(f"  \tTexto: \"{all_texts[0]}\"")
print(f"  \tCodificado: {encoded_texts[0][:15]}...\n")

# 4. ARQUITECTURA DEL MODELO
print("🧠 [4/5] ARQUITECTURA DEL MODELO RNN")
print("-" * 80)
model = SimpleRNNClassifier(config)

print(f"  Matrices de Pesos Inicializadas:")
print(f"  ├─ W_embedding: \t{model.W_embedding.shape} (Input: {config.VOCAB_SIZE}, Output: {config.EMBEDDING_DIM})")
print(f"  ├─ W_xh: \t\t{model.W_xh.shape} (Input: {config.EMBEDDING_DIM}, Output: {config.HIDDEN_DIM})")
print(f"  ├─ W_hh: \t\t{model.W_hh.shape} (Input: {config.HIDDEN_DIM}, Output: {config.HIDDEN_DIM})")
print(f"  └─ W_hy: \t\t{model.W_hy.shape} (Input: {config.HIDDEN_DIM}, Output: {config.OUTPUT_DIM})\n")


## 🎓 Entrenamiento del Modelo (BPTT Manual)

In [ ]:
# 5. ENTRENAMIENTO
print("🎓 [5/5] ENTRENAMIENTO DEL MODELO")
print("-" * 80)

training_losses = []
training_accuracies = []
total_samples = len(all_texts)

for epoch in range(config.EPOCHS):
	total_loss = 0.0
	correct_predictions = 0
	
	# Shuffle data
	indices = list(range(total_samples))
	random.shuffle(indices)
	
	for idx in indices:
		word_indices = encoded_texts[idx]
		label = all_labels[idx]
		
		# Forward pass
		logits, h_sequence, x_sequence = model.forward(word_indices)
		
		# Calcular loss
		loss = model.calculate_loss(logits, label)
		total_loss += loss
		
		# Predicción
		pred_class, _, _ = model.predict(word_indices)
		if pred_class == label:
			correct_predictions += 1
		
		# Backward pass (actualizar pesos)
		model.backward(word_indices, h_sequence, x_sequence, logits, label)
	
	avg_loss = total_loss / total_samples
	accuracy = correct_predictions / total_samples
	
	training_losses.append(avg_loss)
	training_accuracies.append(accuracy)
	
	# Imprimir progreso
	if (epoch + 1) % config.PRINT_EVERY == 0 or epoch == 0:
		print(f"  Época {epoch+1:2d}/{config.EPOCHS} │ " +
			  f"Loss: {avg_loss:.4f} │ " +
			  f"Accuracy: {accuracy:5.1%}")

print("-" * 80)
print("\n")


## ✅ Pruebas de Predicción

Probamos el modelo entrenado con frases no vistas en el conjunto de entrenamiento para evaluar su capacidad de **generalización**.

In [ ]:
print("✅ PRUEBAS DE PREDICCIÓN")
print("=" * 80)

test_cases = [
	("película excelente muy buena recomendada", 1),
	("horrible mala aburrida pérdida de tiempo", 0),
	("actores increíbles historia fascinante", 1),
	("decepcionante y frustrante", 0),
	("me encantó totalmente magnífica", 1),
	("basura no aguanté", 0),
]

correct_test = 0

for test_idx, (test_text, true_label) in enumerate(test_cases, 1):
	word_indices = preprocessor.encode(test_text, max_len=config.MAX_SEQUENCE_LEN)
	pred_label, confidence, probs = model.predict(word_indices)
	
	# Verificar si es correcto
	is_correct = pred_label == true_label
	if is_correct:
		correct_test += 1
	
	# Mostrar resultado
	symbol = "✓" if is_correct else "✗"
	sentiment = "POSITIVO ✨" if pred_label == 1 else "NEGATIVO ❌"
	
	print(f"\n  {symbol} Test {test_idx}: '{test_text}'")
	print(f"  \tPredicción: {sentiment}")
	print(f"  \tConfianza: {confidence*100:.1f}%")
	print(f"  \tP(Neg): {probs[0]:.4f} | P(Pos): {probs[1]:.4f}")

test_accuracy = correct_test / len(test_cases)
print(f"\n  Precisión en pruebas: {correct_test}/{len(test_cases)} ({test_accuracy*100:.1f}%)")
print("=" * 80)
print("\n")


## 🔬 Análisis Detallado de Operaciones Matriciales

La eficiencia de una RNN se mide por el número de **multiplicaciones matriciales** (MACs - Multiply-Accumulate Operations) que realiza, especialmente en las capas ocultas, que se repiten para cada palabra de la secuencia. La aceleración por **GPU/TPU** se basa en la capacidad de paralelizar estas operaciones.

El costo de cálculo de la matriz $\mathbf{A} \mathbf{B}$ de tamaño $(m \times k) \times (k \times n)$ es $O(mkn)$. En nuestro caso, $m=1$ (tamaño de *batch*), por lo que el costo es $O(kn)$.

In [ ]:
# 7. ANÁLISIS DE OPERACIONES MATRICIALES
print("🔬 ANÁLISIS DETALLADO DE OPERACIONES MATRICIALES")
print("=" * 80)

max_seq = config.MAX_SEQUENCE_LEN
emb_dim = config.EMBEDDING_DIM
hidden_dim = config.HIDDEN_DIM
num_samples = len(all_texts)
num_epochs = config.EPOCHS

# Operaciones por paso temporal (Forward Pass)
ops_xh = emb_dim * hidden_dim # x_t @ W_xh
ops_hh = hidden_dim * hidden_dim # h_t @ W_hh
ops_per_step = ops_xh + ops_hh
ops_per_seq = max_seq * ops_per_step

print(f"\n  Operaciones por paso temporal (Solo Forward):")
print(f"  ├─ x_t @ W_xh: {emb_dim} × {hidden_dim} = {ops_xh:,} mults")
print(f"  ├─ h_t @ W_hh: {hidden_dim} × {hidden_dim} = {ops_hh:,} mults")
print(f"  └─ Total por paso: {ops_per_step:,} mults")

print(f"\n  Operaciones por secuencia ({max_seq} palabras):")
print(f"  └─ {max_seq} pasos × {ops_per_step:,} ops/paso = {ops_per_seq:,} ops")

# Operaciones totales de entrenamiento (Forward + Backward)
# El Backward es aproximadamente 2-3 veces más costoso que el Forward
ops_scaling_factor = 3 
total_ops_training = num_samples * num_epochs * ops_per_seq * ops_scaling_factor

print(f"\n  Operaciones totales (entrenamiento estimado, incluyendo BPTT):")
print(f"  └─ {num_samples} muestras × {num_epochs} épocas × {ops_per_seq:,} ops/seq × {ops_scaling_factor}x (BPTT) = {total_ops_training:,} ops")
print(f"  └─ ≈ {total_ops_training / 1e6:.2f}M multiplicaciones/sumas (MACs)")

print("=" * 80)
print("\n")


## 📐 Demostración Explícita de Multiplicación Matricial

El núcleo de la recurrencia es la suma de dos proyecciones lineales, donde $\mathbf{x}_t$ y $\mathbf{h}_{t-1}$ se convierten en el nuevo estado $\mathbf{h}_t$. Esta es la operación más costosa y fundamental.

In [ ]:
print("📐 DEMOSTRACIÓN EXPLÍCITA DE OPERACIÓN MATRICIAL")
print("=" * 80)

# Crear matrices pequeñas para visualización
emb_dim = 4
hidden_dim = 3

x_demo = np.array([[1.0, 0.5, -0.3, 0.2]]) # Embedding (1 × 4)
h_prev_demo = np.array([[0.1, -0.2, 0.3]]) # Estado anterior (1 × 3)
W_xh_demo = np.random.randn(emb_dim, hidden_dim) * 0.1 # (4 × 3)
W_hh_demo = np.random.randn(hidden_dim, hidden_dim) * 0.1 # (3 × 3)

print(f"\n  1️⃣  ENTRADA (x_t): Shape {x_demo.shape}")
print(f"  2️⃣  ESTADO ANTERIOR (h_{{t-1}}): Shape {h_prev_demo.shape}")
print(f"  3️⃣  MATRIZ W_xh (Entrada→Oculto): Shape {W_xh_demo.shape}")
print(f"  4️⃣  MATRIZ W_hh (Oculto→Oculto): Shape {W_hh_demo.shape}")

# Operaciones
z1 = x_demo @ W_xh_demo
z2 = h_prev_demo @ W_hh_demo
z = z1 + z2 # Se omite b_h para simplificar
h_new = np.tanh(z)

print(f"\n  ✖️  OPERACIÓN 1: x_t @ W_xh")
print(f"      ({x_demo.shape}) × ({W_xh_demo.shape}) = ({z1.shape})")
print(f"      Resultado (Proyección de entrada): {np.round(z1, 4)}")

print(f"\n  ✖️  OPERACIÓN 2: h_prev @ W_hh")
print(f"      ({h_prev_demo.shape}) × ({W_hh_demo.shape}) = ({z2.shape})")
print(f"      Resultado (Proyección de estado anterior): {np.round(z2, 4)}")

print(f"\n  ➕ COMBINACIÓN Y ACTIVACIÓN: h_t = tanh(z1 + z2)")
print(f"      NUEVO ESTADO OCULTO (h_t): {np.round(h_new, 4)}")

print("=" * 80)
print("\n")

## 📌 Conclusiones

1.  **Multiplicación Matricial es la Base:** La recurrencia y la transformación de la entrada en cada paso temporal dependen de una o más multiplicaciones matriciales. La eficiencia de un algoritmo RNN (y de todos los modelos modernos de Deep Learning) está directamente ligada a la optimización del hardware para estas operaciones.

2.  **Backpropagation Through Time (BPTT):** El entrenamiento requiere propagar el error hacia atrás no solo a través de las capas, sino también a través del tiempo, lo que implica aún más multiplicaciones matriciales (e.g., $h_{t-1}.T \mathbf{d}h_t$ para $\mathbf{W}_{hh}$).

3.  **Importancia del Hardware:** La estimación de que una GPU puede ser $\approx 50,000\times$ más rápida que una CPU en este tipo de carga ilustra por qué la **aceleración de matrices** es fundamental para el desarrollo de NLP a escala real.